In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import sys
sys.path.append('../code')
import scrape, features
import dataframe_utilities as util

# Update DB & Get Today Games

In [4]:
last_day = pd.to_datetime(pd.read_csv('../data/game_summaries.csv')['date']).max()
get_day = last_day + pd.Timedelta(days=1)
get_day

Timestamp('2019-08-23 00:00:00')

In [5]:
while get_day.date() < pd.datetime.today().date():
    links = scrape.get_game_links(get_day)
    for l in links:
        scrape.process_link(l)
    get_day += + pd.Timedelta(days=1)

BAL201908230 done
CHA201908230 done
CHN201908230 done
CLE201908230 done
HOU201908230 done
LAN201908230 done
MIA201908230 done
MIL201908230 done
MIN201908230 done
NYN201908230 done
PIT201908230 done
SDN201908230 done
SEA201908230 done
SLN201908230 done


In [6]:
test_df = scrape.get_today_games()

In [7]:
test_df

,away_pitcher,away_team_abbr,home_pitcher,home_team_abbr,time,date
0,rossjo01,WSN,quintjo01,CHC,2:20PM,2019-08-24
1,sabatc.01,NYY,gonsoto01,LAD,4:05PM,2019-08-24
2,eflinza01,PHI,yamamjo01,MIA,6:10PM,2019-08-24
3,alvarjo03,TBR,meansjo01,BAL,7:05PM,2019-08-24
4,woodal02,CIN,willitr01,PIT,7:05PM,2019-08-24
5,friedma01,ATL,wheelza01,NYM,7:10PM,2019-08-24
6,jacksed01,DET,gibsoky01,MIN,7:10PM,2019-08-24
7,galleza01,ARI,anderch01,MIL,7:10PM,2019-08-24
8,peterdi01,LAA,mileywa01,HOU,7:10PM,2019-08-24
9,sparkgl01,KCR,clevimi01,CLE,7:10PM,2019-08-24


# Process Stats for Today's Games

## Merge test and train dfs

In [8]:
test_df['is_night_game'] = True
test_df['is_night_game'][test_df['time'].str[:1].astype('int')<5] = False
test_df['is_night_game'][test_df['time'].str[1:2].isin(['0','1'])] = True #for 10,11 PM games
test_df.drop(columns='time', inplace=True)

In [9]:
test_df['is_test'] = True
test_df['home_team_win']=np.nan
test_df['game_id'] = test_df.home_team_abbr + test_df.date.astype('str').str.replace('-','') + '0'
test_df.shape

(15, 9)

In [10]:
df = features.get_game_df()
df['is_test'] = False
df.shape

(24145, 9)

In [11]:
df = pd.concat([df,test_df])
df = df.sort_values(by='date').reset_index(drop=True)
df.shape

(24160, 9)

## Add Features

In [12]:
df = features.add_trueskill_ratings(df)
df = features.add_rest_durations(df)
df.shape

(24160, 18)

In [13]:
#datetime
date = pd.to_datetime(df['date'])
df['season'] = date.dt.year
df['month']=date.dt.month
df['week_num'] = date.dt.week
df['dow']=date.dt.weekday.astype('int')

In [14]:
df['dh_game_no'] = pd.to_numeric(df['game_id'].str[-1:],errors='coerce')
df['date'] = (pd.to_datetime(df['date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') #epoch time

### Add Stats

In [15]:
games = features.get_games()
batting = features.get_batting()
pitching = features.get_pitching()
pitchers = features.get_pitchers()

#### Rolling 10 Day Stats

In [16]:
b_stats = ['batting_avg','leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging']
df = features.add_10RA_rolling(batting, df, b_stats, True, 'batting')

In [17]:
pitching['SO_batters_faced'] = pitching['SO'] / pitching['batters_faced']
pitching['H_batters_faced'] = pitching['H'] / pitching['batters_faced']
pitching['BB_batters_faced'] = pitching['BB'] / pitching['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitching, df, b_stats, True, 'team_pitching')

In [18]:
pitchers['SO_batters_faced'] = pitchers['SO'] / pitchers['batters_faced']
pitchers['H_batters_faced'] = pitchers['H'] / pitchers['batters_faced']
pitchers['BB_batters_faced'] = pitchers['BB'] / pitchers['batters_faced']

# create rolling stat
b_stats = ['earned_run_avg','SO_batters_faced','H_batters_faced','BB_batters_faced']
df = features.add_10RA_rolling(pitchers, df, b_stats, False, 'pitcher')

#### Games Stats

In [19]:
df = features.game_stats(games,df)

In [20]:
df.shape

(24160, 82)

#### Season Stats

In [21]:
batting_stats = ['A', 'AB', 'BB', 'H', 'PA', 'PO', 'R', 'RBI', 'SO', 'batting_avg',
             'leverage_index_avg', 'onbase_perc', 'onbase_plus_slugging', 'pitches', 
             're24_bat', 'slugging_perc', 'strikes_total', 'wpa_bat', 'wpa_bat_neg', 
             'wpa_bat_pos']
df = features.add_season_rolling(batting, df, batting_stats, True,'batting')
df.shape

(24160, 222)

In [22]:
pitching_stats = ['BB', 'ER', 'H', 'HR', 'IP', 'R', 'SO', 'batters_faced',
               'earned_run_avg', 'game_score', 'inherited_runners',
               'inherited_score', 'inplay_fb_total', 'inplay_gb_total', 'inplay_ld',
               'inplay_unk', 'leverage_index_avg', 'pitches', 're24_def',
               'strikes_contact', 'strikes_looking', 'strikes_swinging',
               'strikes_total', 'wpa_def','SO_batters_faced','H_batters_faced',
                'BB_batters_faced']
df = features.add_season_rolling(pitching, df, pitching_stats, True,'team_pitching')
df.shape

(24160, 411)

In [23]:
df = features.add_season_rolling(pitchers, df, pitching_stats, False,'pitcher')
df.shape

(24160, 600)

## Cleanup

In [24]:
df = util.fix_na(df, False)

# Generate Predictions

In [25]:
X_test = df[df.is_test].drop(columns=['is_test'])
cols = pd.read_csv("../data/df_w_features.csv").columns
X_test = X_test[cols]

X_test.drop(columns=['home_team_win','game_id'], inplace=True)

In [26]:
import pickle
encoder, model = pickle.load(open('../data/encoder_model.pk','rb'))
encoder, model

(CatBoostEncoder(cols=['home_team_abbr', 'away_team_abbr', 'home_pitcher',
                       'away_pitcher', 'home_team_season', 'away_team_season'],
                 drop_invariant=False, handle_missing='value',
                 handle_unknown='value', random_state=13, return_df=True,
                 sigma=None, verbose=0),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=None, colsample_bytree=0.65, gamma=0,
               learning_rate=0.3, max_delta_step=0, max_depth=26,
               min_child_weight=18.0, missing=nan, n_estimators=100, n_jobs=1,
               nthread=None, num_boost_round=100, objective='binary:logistic',
               random_state=13, reg_alpha=0.057490000000000006, reg_lambda=1,
               scale_pos_weight=1, seed=None, silent=True,
               subsample=0.6000000000000001, verbosity=None))

In [99]:
X_test = encoder.transform(X_test)

test_df = test_df.sort_values(by=['date','game_id']).reset_index(drop=True)
pred_df = df[df.is_test][['away_pitcher', 'away_team_abbr', 'home_pitcher', 'home_team_abbr']]
pred_df['xgb_probability']= model.predict_proba(X_test)[:,1]

pred_df['xgb_winner']=pred_df.home_team_abbr
pred_df['xgb_winner'][~model.predict(X_test)]=pred_df.away_team_abbr

In [100]:
# get daily odds from covers.com
import requests
from bs4 import BeautifulSoup as bs
html = requests.get('https://www.covers.com/sports/mlb/matchups').text
soup = bs(html)
games = []
for s in soup.findAll('div',{'class':'cmg_matchup_game_box cmg_game_data'}):
    g = {}
    g['home_team_abbr'] = s['data-home-team-shortname-search']
    g['online_odds'] = s['data-game-odd']
    games.append(g)
odds = pd.DataFrame(games)

# merge in the odds
pred_df = pd.merge(left=pred_df, right=odds, on='home_team_abbr')
pred_df['online_odds']=pred_df['online_odds'].astype('int')
pred_df['online_odds'][pred_df.xgb_probability<0.5] = -pred_df['online_odds'] #convert odds to pred winner odds (not home team odds)

In [101]:
#https://www.bettingexpert.com/en-au/learn/understanding-betting-odds/how-to-convert-odds
pred_df['online_proba'] = -pred_df['online_odds']/(-pred_df['online_odds']+100)
pred_df['online_proba'][pred_df['online_odds']>0] = 100/(pred_df['online_odds']+100)
online_abs = np.abs(pred_df['online_proba']-0.5)
xgb_abs = np.abs(pred_df['xgb_probability']-0.5)
pred_df['pred_diff_percent'] = online_abs-xgb_abs

In [102]:
teams = pd.read_csv("../data/teams.csv")
pred_df = pd.merge(left=pred_df, right=teams, left_on='xgb_winner',right_on='Abbr')

pred_df['predicted_winner'] = pred_df['Team']
pred_df.drop(columns=['xgb_winner','Abbr','Team'], inplace=True)

In [103]:
pred_df

,away_pitcher,away_team_abbr,home_pitcher,home_team_abbr,xgb_probability,online_odds,online_proba,pred_diff_percent,predicted_winner
0,woodal02,CIN,willitr01,PIT,0.324543,-100,0.500000,-0.175457,Reds
1,bumgama01,SFG,bassich01,OAK,0.549485,-148,0.596774,0.047289,Athletics
2,friedma01,ATL,wheelza01,NYM,0.604298,-110,0.523810,-0.080488,Mets
3,jacksed01,DET,gibsoky01,MIN,0.527489,-333,0.769053,0.241564,Twins
4,galleza01,ARI,anderch01,MIL,0.548487,-125,0.555556,0.007068,Brewers
5,peterdi01,LAA,mileywa01,HOU,0.560229,-215,0.682540,0.122311,Astros
6,sabatc.01,NYY,gonsoto01,LAD,0.686344,-141,0.585062,-0.101282,Dodgers
7,sparkgl01,KCR,clevimi01,CLE,0.759646,-385,0.793814,0.034169,Indians
8,allarko01,TEX,novaiv01,CHW,0.321013,136,0.423729,-0.102715,Rangers
9,rossjo01,WSN,quintjo01,CHC,0.672252,-175,0.636364,-0.035889,Cubs
